# **Finetuning GPT2 using HuggingFace and Tensorflow**

In this colab notebook we set up a simple outline of how you can use Huggingface to fine tune a gpt2 model on finance titles to generate new possible headlines. This notebook uses the hugginface finefuning scripts and then uses the TensorFlow version of the genreated models.

First begin setup by cloning transformers repo. We need to store the training script locally since there isnt an easier way to train tf based gpt2 models as far as I can see.

In [ ]:
#Clone the transformers repo into the notebook
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 93421, done.
remote: Counting objects: 100% (308/308), done.
remote: Compressing objects: 100% (183/183), done.
remote: Total 93421 (delta 155), reused 217 (delta 103), pack-reused 93113
Receiving objects: 100% (93421/93421), 85.88 MiB | 24.43 MiB/s, done.
Resolving deltas: 100% (68449/68449), done.


In [ ]:
# Clone should now be in the machine
!ls

output	sample_data  total.txt	transformers


Check to see what gpu we were granted. For Colab Pro it will vary between a Tesla V100 or P100. For normal colab it should be a k80

In [ ]:
!nvidia-smi

Fri May  6 05:25:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Change directory location to be in the examples folder and then install any requirements

In [ ]:
import os
os.chdir("/content/transformers")
os.chdir("/content/transformers/examples/pytorch/language-modeling")
!ls

README.md	  run_clm_no_trainer.py  run_mlm_no_trainer.py	run_plm.py
requirements.txt  run_clm.py		 run_mlm.py


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 79 kB 618 kB/s 
     |████████████████████████████████| 325 kB 33.3 MB/s 
     |████████████████████████████████| 1.2 MB 42.0 MB/s 
     |████████████████████████████████| 1.1 MB 35.6 MB/s 
     |████████████████████████████████| 136 kB 50.7 MB/s 
     |████████████████████████████████| 212 kB 47.9 MB/s 
     |████████████████████████████████| 77 kB 6.4 MB/s 
     |████████████████████████████████| 127 kB 47.4 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 144 kB 38.7 MB/s 
     |████████████████████████████████| 271 kB 49.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium

In [ ]:
!ls

README.md	  run_clm_no_trainer.py  run_mlm_no_trainer.py	run_plm.py
requirements.txt  run_clm.py		 run_mlm.py


In [ ]:
!pip install pyarrow --upgrade

     |████████████████████████████████| 26.7 MB 1.5 MB/s 
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 6.0.1
    Uninstalling pyarrow-6.0.1:
      Successfully uninstalled pyarrow-6.0.1


In [ ]:
import os
os.chdir("/content/transformers")
os.chdir("/content/transformers/examples/pytorch/language-modeling")


In [ ]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-euv7_k0q
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-euv7_k0q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 46.2 MB/s 
  Created wheel for transformers: filename=transformers-4.19.0.dev0-py3-none-any.whl size=4101907 sha256=243376e4549bda1737872c8fddfaa6d899539f537259db52feaa89d9deb5a1f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-rai2ycx4/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Set up data from a text file in the format <|title|> some data <|endoftext|> and split into training and eval sets.

In [ ]:
#"""
#Now load the data line by line
#"""
from sklearn.model_selection import train_test_split

with open('/content/total.txt', 'r') as data: 
  dataset = ["<|title|>" + x.strip() for x in data.readlines()]

train, eval = train_test_split(dataset, train_size=.9, random_state=2020)
print("training size:" + str(len(train)))
print("Evaluation size: " + str(len(eval)))

with open('train_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(train))

with open('eval_tmp.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(eval))

training size:127378
Evaluation size: 14154


The script below will fine tune GPT2 on your text data that you setup above. This training step will take anywhre from tens of minutes to hours depending on how large your training set is, how many epochs you intend to train on, and if you are using colab or colab pro. We utilize mixed precision in this model to shave off some training time. For a large data set I was using for another experiment it saved us over 30 mins in training time.

# **Using the model**
Next lets take our model we just trained and use it to generate some text! We will import the Tensorflow version of the gpt2 language model and set the from_pt flag to True. Then we load a pretrained tokenizer from huggingface. This may take some time to download the tokenizer data.

In [ ]:
!ls


eval_tmp.txt	  run_clm_no_trainer.py  run_mlm.py
README.md	  run_clm.py		 run_plm.py
requirements.txt  run_mlm_no_trainer.py  train_tmp.txt


In [ ]:
!python run_clm.py \
--model_type gpt2 \
--model_name_or_path gpt2 \
--train_file "train_tmp.txt" \
--do_train \
--validation_file "eval_tmp.txt" \
--do_eval \
--per_gpu_train_batch_size 1 \
--save_steps -1 \
--num_train_epochs 5 \
--fp16 \
--output_dir="</content/output>"

05/06/2022 05:27:20 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
05/06/2022 05:27:20 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN

In [ ]:
# setup imports to use the model
from transformers import TFGPT2LMHeadModel
from transformers import GPT2Tokenizer
#/content/transformers/examples/pytorch/language-modeling/</content/output>/runs
model = TFGPT2LMHeadModel.from_pretrained("/content/transformers/examples/pytorch/language-modeling/model/content/output", from_pt=True)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.7.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'lm_head.weight', 'transformer.h.3.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.5.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Encoding sample text is now extremely simple using the pretrained tokenizer.

In [ ]:
input_ids = tokenizer.encode("I love deep learning", return_tensors='tf')


In [ ]:
# the tf tensor object
input_ids[0]

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([  40, 1842, 2769, 4673], dtype=int32)>

In [ ]:
outputs = model.generate(input_ids, max_length=200, num_return_sequences=5, top_k=125, do_sample=True)
for i, beam in enumerate(outputs ):
  print("{}: {}".format(i,tokenizer.decode(beam, skip_special_tokens=True)))
  print()


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


0: I love deep learning, that's all i hear

1: I love deep learning

2: I love deep learning, not fancy words, but a little bit of every color

3: I love deep learning you're my my favorite, you make me shine

4: I love deep learning, all our friends in the game



In [ ]:
generated_text_samples = model.generate(
    input_ids, 
    max_length=250,  
    num_return_sequences=5,
    no_repeat_ngram_size=2,
    repetition_penalty=1.5,
    top_p=0.92,
    temperature=.85,
    do_sample=True,
    top_k=125,
    early_stopping=True
)


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [ ]:
#Print output for each sequence generated above
beam_output=generated_text_samples
for i, beam in enumerate(beam_output):
  print("{}: {}".format(i,tokenizer.decode(beam, skip_special_tokens=True)))
  print()

0: I love deep learning, i want to learn it

1: I love deep learning

2: I love deep learning

3: I love deep learning but when it comes to what i'm trying

4: I love deep learning

